In [ ]:
from dateutil.relativedelta import relativedelta
import requests
import pandas as pd
from datetime import datetime
import pytz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Se si usa solo un sottoinsieme (chosen airports) conviene usare questo codice così non fa doppioni su dataset, perche codice fa andata ritorno
chosen_airports = ['BLQ', 'BVA']

import itertools
routes = []
for subset in itertools.combinations(chosen_airports, 2):
    routes.append(subset)
print(routes)

[('BLQ', 'BVA')]


In [ ]:
now = datetime.now() # current date and time

tz_Rome = pytz.timezone('Europe/Rome')
datetime_Rome = datetime.now(tz_Rome)
hour = datetime_Rome.strftime("%H:%M")
today_date = now.strftime("%m-%d-%Y")
date_time = today_date + '-' + hour

In [ ]:
# Get flights information for the airports chosen

today = datetime.today()
master_df = pd.DataFrame(columns = ['Flight Code', 'Origin','Destination','Period','Date','Departure Time', 'Arrival Time', 'Price','Published Price', 'Tracking Date', 'Tracking Hour'])
date_dict = {}

for keys, values in routes:
    for i in range(34):
        day = today + relativedelta(days=+i)
        date_dict[str(i) + 'day'] = day
    two_months = today + relativedelta(months=+2)
    date_dict['Two Months'] = two_months
    three_months = today + relativedelta(months=+3)
    date_dict['Three Months'] = three_months
    four_month = today + relativedelta(months=+4)
    date_dict['Four Months'] = four_month
    for keys2, values2  in date_dict.items():
        date_in = values2.strftime('%Y-%m-%d')
        date_out = values2 + relativedelta(days=+0)
        date_out = date_out.strftime('%Y-%m-%d')
        url = 'https://www.ryanair.com/api/booking/v4/en-gb/availability?ADT=1&CHD=0&DateIn={}&DateOut={}&Destination={}&Disc=0&INF=0&Origin={}&TEEN=0&promoCode=&IncludeConnectingFlights=false&RoundTrip=false&ToUs=AGREED'.format(date_in, date_out, values, keys)
        content = str(requests.get(url).content)
        flight_keys, prices, published_prices = ([] for i in range(3)) 
        for i, _ in enumerate(content):
            needle1 = '"flightKey":"'
            needle2 = '"amount":'
            needle3 = '"publishedFare":'
            if content[i:i + len(needle1)] == needle1:
                flight_keys.append(content[i+13:i+200].split('~~","',1)[0])
            if content[i:i + len(needle2)] == needle2:
                prices.append(content[i+9:i+18].split(',',1)[0])
            if content[i:i + len(needle3)] == needle3:
                published_prices.append(content[i+16:i+25].split(',',1)[0])
        if len(flight_keys) == len(published_prices) == len(prices):
            for i in range(0, len(flight_keys)):
                flight_code = flight_keys[i][:7]
                departure_time = flight_keys[i][-26:-21]
                arrival_time = flight_keys[i][-5:]
                date  = flight_keys[i][15:26]
                if flight_keys[i][-20:-17] == values:
                    origin = keys
                    destination = values
                else:
                    origin = values
                    destination = keys
                price = prices[i]
                published_price = published_prices[i]
                row = [flight_code, origin,destination,keys2,date,departure_time,arrival_time,price,published_price, today_date, hour]
                master_df.loc[len(master_df)] = row
print(master_df)

   Flight Code Origin Destination        Period         Date Departure Time  \
0      FR~4303    BLQ         BVA          1day  05/23/2022           06:10   
1      FR~4303    BLQ         BVA          2day  05/24/2022           07:05   
2      FR~4303    BLQ         BVA          3day  05/25/2022           06:20   
3      FR~4303    BLQ         BVA          4day  05/26/2022           06:15   
4      FR~4303    BLQ         BVA          5day  05/27/2022           06:20   
5      FR~4303    BLQ         BVA          6day  05/28/2022           06:05   
6      FR~4303    BLQ         BVA          7day  05/29/2022           09:15   
7      FR~4303    BLQ         BVA          8day  05/30/2022           06:10   
8      FR~4303    BLQ         BVA          9day  05/31/2022           07:05   
9      FR~4303    BLQ         BVA         10day  06/01/2022           06:20   
10     FR~4303    BLQ         BVA         11day  06/02/2022           06:15   
11     FR~4303    BLQ         BVA         12day  06/

In [ ]:
master_df.to_csv('/content/drive/MyDrive/AI_Industry/OtherFlights/BLQBVA/data'+date_time+'.csv')